In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import cartopy.crs as ccrs
import time

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro 1/'
dir3='/Volumes/Extreme Pro/'

In [ ]:
yrs=np.arange(1980,2020,1)

ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
#ds=xr.open_dataset(dir_data+'jra55_gridareas_125.nc')
areas=ds['cell_area']/1e6

if 1==1:
    ivt_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    ivt_max_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    iwv_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    iwv_max_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    s_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    s_max_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    mfc_array=[[],[],[],[],[],[],[],[],[],[],[],[]]
    mfc_max_array=[[],[],[],[],[],[],[],[],[],[],[],[]]

    ivt_list=[]
    ivt_max_list=[]
    iwv_list=[]
    iwv_max_list=[]
    s_list=[]
    s_max_list=[]
    mfc_list=[]
    mfc_max_list=[]

for i in range(0,len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==1:
        ds=xr.open_dataset(dir_data+'era5mk.ar.labels.model.'+str(year)+'.nc')
        labeled=ds['ar_labeled']

        if 1==1:
            ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
            ivt=ds['ivt'][:,:,:]
            ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
            iwv=ds['iwv'][:,:,:]
            ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
            u=ds['u']
            ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
            v=ds['v']
            s=np.sqrt(u**2+v**2)
            ds=xr.open_dataset(dir2+'era_5_mfcinterp_'+str(year)+'.nc')#era5 is divergence
            mfc=ds['mfc'][:,:,:]*-1.
        if 1==0:
            ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
            ivt=ds['ivt'][0::2,:,:]
            ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
            iwv=ds['iwv'][0::2,:,:]
            ds=xr.open_dataset(dir3+'merra2.u850.6hr.'+str(year)+'.nc')
            u=ds['u']
            ds=xr.open_dataset(dir3+'merra2.v850.6hr.'+str(year)+'.nc')
            v=ds['v']
            s=np.sqrt(u**2+v**2)

            ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
            mfc=ds['mfc'][0::2,:,:]
        if 1==0:
            ds=xr.open_dataset(dir3+'jra55_ivt_'+str(year)+'.nc')
            ivt=ds['ivt'].interp(lat=labeled.lat.values,lon=labeled.lon.values)
            ds=xr.open_dataset(dir3+'jra55_iwv_'+str(year)+'.nc')
            iwv=ds['iwv'].interp(lat=labeled.lat.values,lon=labeled.lon.values)
            ds=xr.open_dataset(dir3+'jra55_u850_'+str(year)+'.nc')
            u=ds['u'].interp(lat=labeled.lat.values,lon=labeled.lon.values)
            ds=xr.open_dataset(dir3+'jra55_v850_'+str(year)+'.nc')
            v=ds['v'].interp(lat=labeled.lat.values,lon=labeled.lon.values)
            s=np.sqrt(u**2+v**2)
        
        ivt['lon']=labeled.lon.values
        iwv['lon']=labeled.lon.values
        s['lon']=labeled.lon.values
        mfc['lon']=labeled.lon.values
        
        ivt['lat']=labeled.lat.values
        iwv['lat']=labeled.lat.values
        s['lat']=labeled.lat.values
        mfc['lat']=labeled.lat.values

        if i==0:
            weights=np.cos(np.deg2rad(ds.lat))
        
        ivt_list_year=[]
        ivt_max_list_year=[]
        iwv_list_year=[]
        iwv_max_list_year=[]
        s_list_year=[]
        s_max_list_year=[]
        mfc_list_year=[]
        mfc_max_list_year=[]
        
        ivt_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        ivt_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        iwv_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        s_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        mfc_max_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        
        
        num_list_year=[]
        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
        #if 1==0:
            if j%100==0:
                print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)
            
            ivt_sel=ivt[j,:,:]
            iwv_sel=iwv[j,:,:]
            s_sel=s[j,:,:]
            mfc_sel=mfc[j,:,:]

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                area_feature=area_feature.sum(skipna=True).values.tolist()
                #print(area_feature)
                if area_feature>0:
                    ivt_sel1=ivt_sel.where(sel_feature>0)
                    ivt_weighted=ivt_sel1.weighted(weights)
                    ivt_mean=ivt_weighted.mean(skipna=True).values.tolist()
                    ivt_max=ivt_sel1.max().values.tolist()

                    iwv_sel1=iwv_sel.where(sel_feature>0)
                    iwv_weighted=iwv_sel1.weighted(weights)
                    iwv_mean=iwv_weighted.mean(skipna=True).values.tolist()
                    iwv_max=iwv_sel1.max().values.tolist()

                    s_sel1=s_sel.where(sel_feature>0)
                    s_weighted=s_sel1.weighted(weights)
                    s_mean=s_weighted.mean(skipna=True).values.tolist()
                    s_max=s_sel1.max().values.tolist()

                    mfc_sel1=mfc_sel.where(sel_feature>0)
                    mfc_weighted=mfc_sel1.weighted(weights)
                    mfc_mean=mfc_weighted.mean(skipna=True).values.tolist()
                    mfc_max=mfc_sel1.max().values.tolist()

                    ivt_list_year.append(ivt_mean)
                    ivt_max_list_year.append(ivt_max)
                    iwv_list_year.append(iwv_mean)
                    iwv_max_list_year.append(iwv_max)
                    s_list_year.append(s_mean)
                    s_max_list_year.append(s_max)
                    mfc_list_year.append(mfc_mean)
                    mfc_max_list_year.append(mfc_max)
                    
                    if 1==0:
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(ivt_sel.lon.values,ivt_sel.lat.values,ivt_sel1,vmin=0,vmax=800,cmap=plt.cm.seismic)
                        plt.show()

                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(ivt_sel.lon.values,ivt_sel.lat.values,iwv_sel1,vmin=0,vmax=60,cmap=plt.cm.seismic)
                        plt.show()
                        
                        fig=plt.figure(figsize=(18,12))
                        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        ax.coastlines(resolution='10m')
                        ax.pcolormesh(ivt_sel.lon.values,ivt_sel.lat.values,s_sel1,vmin=0,vmax=30,cmap=plt.cm.seismic)
                        plt.show()
                        
                        #fig=plt.figure(figsize=(18,12))
                        #ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                        #ax.coastlines(resolution='10m')
                        #ax.pcolormesh(ivt_sel.lon.values,ivt_sel.lat.values,mfc_sel1,vmin=-4e-4,vmax=4e-4,cmap=plt.cm.seismic)
                        #plt.show()

                    ivt_array_year[month-1].append(ivt_mean)
                    ivt_max_array_year[month-1].append(ivt_max)
                    iwv_array_year[month-1].append(iwv_mean)
                    iwv_max_array_year[month-1].append(iwv_max)
                    s_array_year[month-1].append(s_mean)
                    s_max_array_year[month-1].append(s_max)
                    mfc_array_year[month-1].append(mfc_mean)
                    mfc_max_array_year[month-1].append(mfc_max)
                    
                    num_day=num_day+1
                    
                    if 1==0:
                        
                        print('IVT:'+str(ivt_mean)+','+str(ivt_max))
                        print('IWV:'+str(iwv_mean)+','+str(iwv_max))
                        print('S:'+str(s_mean)+','+str(s_max))
                        #print('IVTDIV:'+str(ivtdiv_mean)+','+str(ivtdiv_max))
                        #print('MFC:'+str(mfc_mean)+','+str(mfc_max))
                        plt.show()

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)
            
            
            #sys.exit()

        s_list_year=[x for x in s_list_year if x>=0]
        s_max_list_year=[x for x in s_max_list_year if x>=0]
        
        mean_ivt=float(sum(ivt_list_year))/float(len(ivt_list_year))
        max_ivt=float(sum(ivt_max_list_year))/float(len(ivt_max_list_year))
        mean_iwv=float(sum(iwv_list_year))/float(len(iwv_list_year))
        max_iwv=float(sum(iwv_max_list_year))/float(len(iwv_max_list_year))
        mean_s=float(sum(s_list_year))/float(len(s_list_year))
        max_s=float(sum(s_max_list_year))/float(len(s_max_list_year))
        mean_mfc=float(sum(mfc_list_year))/float(len(mfc_list_year))
        max_mfc=float(sum(mfc_max_list_year))/float(len(mfc_max_list_year))
        
        ivt_list.append(mean_ivt)
        ivt_max_list.append(max_ivt)
        iwv_list.append(mean_iwv)
        iwv_max_list.append(max_iwv)
        s_list.append(mean_s)
        s_max_list.append(max_s)
        mfc_list.append(mean_mfc)
        mfc_max_list.append(max_mfc)
        
        
        for j in range(len(nums_array_year)):
            ivt_array[j].append(float(sum(ivt_array_year[j]))/float(len(ivt_array_year[j])))
            ivt_max_array[j].append(float(sum(ivt_max_array_year[j]))/float(len(ivt_max_array_year[j])))
        
            iwv_array[j].append(float(sum(iwv_array_year[j]))/float(len(iwv_array_year[j])))
            iwv_max_array[j].append(float(sum(iwv_max_array_year[j]))/float(len(iwv_max_array_year[j])))
        
            s_array[j].append(float(sum(s_array_year[j]))/float(len(s_array_year[j])))
            s_max_array[j].append(float(sum(s_max_array_year[j]))/float(len(s_max_array_year[j])))

            mfc_array[j].append(float(sum(mfc_array_year[j]))/float(len(mfc_array_year[j])))
            mfc_max_array[j].append(float(sum(mfc_max_array_year[j]))/float(len(mfc_max_array_year[j])))

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(yrs,mfc_max_list)
plt.show()

In [ ]:
dk=xr.Dataset()

dk['ivt_array']=(('months','years'),np.asarray(ivt_array))
dk['ivt_max_array']=(('months','years'),np.asarray(ivt_max_array))
dk['iwv_array']=(('months','years'),np.asarray(iwv_array))
dk['iwv_max_array']=(('months','years'),np.asarray(iwv_max_array))
dk['s_array']=(('months','years'),np.asarray(s_array))
dk['s_max_array']=(('months','years'),np.asarray(s_max_array))
dk['mfc_array']=(('months','years'),np.asarray(mfc_array))
dk['mfc_max_array']=(('months','years'),np.asarray(mfc_max_array))

dk['ivt_list']=(('years'),ivt_list)
dk['ivt_max_list']=(('years'),ivt_max_list)
dk['iwv_list']=(('years'),iwv_list)
dk['iwv_max_list']=(('years'),iwv_max_list)
dk['s_list']=(('years'),s_list)
dk['s_max_list']=(('years'),s_max_list)
dk['mfc_list']=(('years'),mfc_list)
dk['mfc_max_list']=(('years'),mfc_max_list)

dk.coords['years']=np.arange(1980,2020,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_synoptic_era5mk.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_synoptic_era5mk.nc',mode='w',format='NETCDF4')